In [1]:
%pwd

'/home/rakib/Learning/AI-medical-chatbot/research'

In [2]:
import os
os.chdir("../")
%pwd

'/home/rakib/Learning/AI-medical-chatbot'

In [54]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [55]:
# extract data from pdf
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader) # load all pdf files in the directory and extract text by PyPDFLoader
    documents = loader.load()
    return documents

In [57]:
extracted_data = load_pdf_file(data="data/")

In [58]:
# split text into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20) # split text into chunks of 1000 characters
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [59]:
text_chunks=text_split(extracted_data)

In [60]:
from langchain.embeddings import HuggingFaceEmbeddings

# download the embedding from huggingface
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [61]:
embedding = download_hugging_face_embeddings()

In [62]:
query_result = embedding.embed_query("What is the capital of France?")
print(len(query_result))

384


In [63]:
from dotenv import load_dotenv
load_dotenv()

True

In [64]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [ ]:
# pinecone 
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "ai-medicale-chatbot"

#FIXME: check if index already exists

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "ai-medicale-chatbot",
    "metric": "cosine",
    "host": "ai-medicale-chatbot-pfixa4j.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [67]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [68]:
# embed each chunk and upsert each chunk into the pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    embedding = embedding,
    index_name = index_name,
)

In [69]:
# load existing index
docsearch = PineconeVectorStore.from_existing_index(
    embedding = embedding,
    index_name = index_name,
)

In [70]:
retriver = docsearch.as_retriever(search_type="similarity",search_kwargs={"k": 3}) # retrive the top 3 most relevant documents

In [ ]:
# retrived_docs = retriver.get_relevant_documents("What is Acne?")

/tmp/ipykernel_80692/45650185.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrived_docs = retriver.get_relevant_documents("What is Acne?")


In [ ]:
# retrived_docs

[Document(id='5a781008-30c3-4afe-b444-740dcf59b121', metadata={'creationdate': '2004-12-18T17:52:16-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T18:38:56-06:00', 'page': 685.0, 'page_label': '686', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data/Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf', 'total_pages': 940.0}, page_content='rare disorder characterized by an abnormal increase in\nthe number of mature red cells in the blood.\nGALE ENCYCLOPEDIA OF MEDICINE 2 2981\nSecondary polycythemia\nThis young boy is afflicted with seborrheic dermatitis.(Cus-\ntom Medical Stock Photo. Reproduced by permission.)\nKEY TERMS\nAcne—A chronic inflammation of the sebaceous\nglands that manifests as blackheads, whiteheads,\nand/or pustules on the face or trunk.\nPsoriasis—A skin disorder of chronic, itchy scaling'),
 Document(id='c89810ad-ce72-4e96-8973-6ddcb6e53fe2', metadata={'creationdate': '2004-12-18T17:52:16-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T18:38:56-06:00', 'page'

In [79]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4) # set the temperature to 0.4 for more deterministic results

In [80]:
from langchain.chains import  create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering task. "
    "Use the following pieces of context to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "Use three sentences maximum to keep the answer concise. "
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [81]:
questions_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, questions_answer_chain)

In [82]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response['result'])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}